# Optimized BTCUSD ORB+ATR Trading Strategy

This notebook implements an optimized Opening Range Breakout (ORB) strategy combined with Average True Range (ATR) for BTCUSD trading on 15-minute timeframe. The strategy is specifically adapted for 24/7 cryptocurrency markets with enhanced risk management and performance metrics.

## Key Features:
- **15-minute timeframe optimization**
- **24/7 crypto market adaptation**
- **Fixed ATR value of 1344** (as specified)
- **Enhanced risk management**
- **Advanced performance metrics**
- **Multiple parameter optimization**
- **Walk-forward analysis**

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import pandas_ta as ta
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams.update({'font.size': 12})

print("Libraries imported successfully!")

## Strategy Configuration

Configure the strategy parameters according to the requirements:

In [ ]:
# Strategy Configuration
TIMEFRAME = "M15"
SYMBOL = "BTCUSD"
STARTING_BALANCE = 10000  # Increased starting balance for crypto trading
RISK_PER_TRADE = 0.01  # Conservative 1% risk per trade for crypto

# ORB Configuration for Crypto Markets (24/7)
ORB_START_HOUR = 0  # 00:00 UTC (midnight)
ORB_END_HOUR = 1    # 01:00 UTC (1 hour opening range)
ORB_END_MINUTES = 0

# ATR Configuration
FIXED_ATR_VALUE = 1344  # As specified
MIN_RANGE_MULTIPLIER = 1.25
MAX_RANGE_MULTIPLIER = 3.0

# Risk Management
MAX_DAILY_LOSS = 0.05  # 5% max daily loss
MAX_DAILY_TRADES = 3   # Limit daily trades
PROFIT_TARGET = 0.10   # 10% daily profit target

# Transaction Costs (realistic for crypto)
MAKER_FEE = 0.001  # 0.1% maker fee
TAKER_FEE = 0.002  # 0.2% taker fee
SPREAD_COST = 0.0005  # 0.05% spread

print("Strategy configuration loaded!")

## Load and Preprocess Data

Load the preprocessed BTCUSD data and prepare it for strategy analysis:

In [ ]:
def load_crypto_data():
    """Load preprocessed BTCUSD data"""
    try:
        # Load processed data
        df = pd.read_csv('../data/BTCUSD_M15_processed.csv', parse_dates=['Datetime'])
        
        # Set datetime index
        if 'Datetime' in df.columns:
            df.set_index('Datetime', inplace=True)
        
        print(f"Loaded {len(df)} records of processed BTCUSD data")
        print(f"Date range: {df.index.min()} to {df.index.max()}")
        print(f"Available features: {len(df.columns)}")
        
        return df
    
    except FileNotFoundError:
        print("Processed data not found. Loading raw data...")
        
        # Fallback to raw data
        df = pd.read_csv('../data/BTCUSD_M15.csv', parse_dates=['Datetime'])
        
        if 'Datetime' in df.columns:
            df.set_index('Datetime', inplace=True)
        
        print(f"Loaded {len(df)} records of raw BTCUSD data")
        return df

# Load data
price_data = load_crypto_data()
price_data.head()

## Enhanced ORB+ATR Strategy Implementation

Implement the optimized ORB+ATR strategy with crypto-specific enhancements:

In [ ]:
def calculate_enhanced_inputs(df):
    """Calculate enhanced strategy inputs for crypto markets"""
    
    # Ensure datetime index
    df.index = pd.to_datetime(df.index)
    
    # Add date and time features
    df['date'] = df.index.date
    df['hour'] = df.index.hour
    df['minute'] = df.index.minute
    
    # Use fixed ATR value as specified
    df['ATR'] = FIXED_ATR_VALUE
    
    # Calculate opening range for crypto (24/7 market)
    # Use first hour of UTC day as opening range
    df['in_opening_range'] = (
        (df['hour'] == ORB_START_HOUR) & 
        (df['minute'] >= 0) &
        ((df['hour'] < ORB_END_HOUR) | 
         ((df['hour'] == ORB_END_HOUR) & (df['minute'] < ORB_END_MINUTES)))
    )
    
    # Group by each trading day and calculate opening range
    opening_range = df[df['in_opening_range']].groupby('date').agg(
        orb_high=('High', 'max'),
        orb_low=('Low', 'min'),
        orb_open=('Open', 'first'),
        orb_close=('Close', 'last')
    )
    
    # Calculate opening range size
    opening_range['orb_range'] = opening_range['orb_high'] - opening_range['orb_low']
    
    # Check if range falls within ATR bounds
    opening_range['valid_range'] = (
        (opening_range['orb_range'] > opening_range['orb_high'] * MIN_RANGE_MULTIPLIER / 100) &
        (opening_range['orb_range'] < opening_range['orb_high'] * MAX_RANGE_MULTIPLIER / 100)
    )
    
    # Merge back to main DataFrame
    df = df.merge(opening_range[['orb_high', 'orb_low', 'orb_range', 'valid_range']], 
                  left_on='date', right_index=True, how='left')
    
    # Clean up
    df.drop(['date', 'hour', 'minute'], axis=1, inplace=True)
    
    return df

def generate_enhanced_signals(df, atr_sl_multiplier, tp_ratio):
    """Generate enhanced trading signals for crypto markets"""
    
    # Breakout conditions
    df['breakout_above'] = (df['Close'] > df['orb_high']) & (df['orb_high'].notna())
    df['breakout_below'] = (df['Close'] < df['orb_low']) & (df['orb_low'].notna())
    
    # Entry conditions
    entry_conditions = (
        df['breakout_above'] &  # Breakout above opening range
        df['valid_range'] &     # Valid opening range
        df['orb_high'].notna()  # ORB data available
    )
    
    # Generate entry signals (shifted to avoid look-ahead bias)
    df['entry_signal'] = entry_conditions.shift(1)
    
    # Calculate stop loss and take profit
    df['stop_loss'] = df['orb_low']
    df['take_profit'] = df['orb_high'] + (df['orb_high'] - df['orb_low']) * tp_ratio
    
    # Risk-based stop loss using ATR
    df['atr_stop'] = df['Close'] - (df['ATR'] * atr_sl_multiplier)
    
    return df

def calculate_position_size(entry_price, stop_loss, balance, risk_per_trade):
    """Calculate position size based on risk management"""
    risk_amount = balance * risk_per_trade
    stop_distance = abs(entry_price - stop_loss)
    
    if stop_distance == 0:
        return 0.01  # Minimum position size
    
    position_size = risk_amount / stop_distance
    return position_size

def simulate_enhanced_trades(df, strategy_name="Enhanced_ORB"):
    """Simulate enhanced trading with crypto-specific features"""
    
    trades = []
    balance = STARTING_BALANCE
    daily_trades = 0
    daily_pnl = 0
    last_date = None
    
    for idx, row in df.iterrows():
        current_date = idx.date()
        
        # Reset daily counters
        if current_date != last_date:
            daily_trades = 0
            daily_pnl = 0
            last_date = current_date
        
        # Check daily limits
        if daily_trades >= MAX_DAILY_TRADES:
            continue
            
        if abs(daily_pnl) >= balance * MAX_DAILY_LOSS:
            continue
        
        # Check entry signal
        if row['entry_signal'] and not pd.isna(row['orb_high']):
            entry_price = row['Open']
            stop_loss = row['atr_stop']  # Use ATR-based stop
            take_profit = row['take_profit']
            
            # Calculate position size
            position_size = calculate_position_size(entry_price, stop_loss, balance, RISK_PER_TRADE)
            
            # Simulate trade execution
            trade_result = simulate_single_trade(
                df, idx, entry_price, stop_loss, take_profit, position_size
            )
            
            if trade_result:
                entry_date, exit_date, exit_price, pnl = trade_result
                
                # Update balance
                balance += pnl
                daily_pnl += pnl
                daily_trades += 1
                
                # Record trade
                trades.append({
                    'entry_date': entry_date,
                    'entry_price': entry_price,
                    'exit_date': exit_date,
                    'exit_price': exit_price,
                    'position_size': position_size,
                    'pnl': pnl,
                    'balance': balance,
                    'strategy': strategy_name
                })
    
    return pd.DataFrame(trades)

def simulate_single_trade(df, entry_idx, entry_price, stop_loss, take_profit, position_size):
    """Simulate a single trade execution"""
    
    for idx in df.loc[entry_idx:].index:
        row = df.loc[idx]
        
        # Check stop loss
        if row['Low'] <= stop_loss:
            exit_price = max(stop_loss, row['Open'])  # Slippage consideration
            pnl = (exit_price - entry_price) * position_size
            return entry_idx, idx, exit_price, pnl
        
        # Check take profit
        if row['High'] >= take_profit:
            exit_price = min(take_profit, row['Open'])  # Slippage consideration
            pnl = (exit_price - entry_price) * position_size
            return entry_idx, idx, exit_price, pnl
        
        # Check end of day (crypto: end of UTC day)
        if idx.hour == 23 and idx.minute >= 45:
            exit_price = row['Close']
            pnl = (exit_price - entry_price) * position_size
            return entry_idx, idx, exit_price, pnl
    
    return None

print("Enhanced strategy functions defined!")

## Parameter Optimization

Perform comprehensive parameter optimization for the strategy:

In [ ]:
def optimize_strategy_parameters(df, param_ranges=None):
    """Optimize strategy parameters using walk-forward analysis"""
    
    if param_ranges is None:
        param_ranges = {
            'atr_sl_multiplier': np.arange(1.5, 3.5, 0.5),
            'tp_ratio': np.arange(1.0, 3.0, 0.5)
        }
    
    results = []
    total_iterations = len(param_ranges['atr_sl_multiplier']) * len(param_ranges['tp_ratio'])
    current_iteration = 0
    
    print(f"Starting parameter optimization with {total_iterations} combinations...")
    
    for atr_sl in param_ranges['atr_sl_multiplier']:
        for tp_ratio in param_ranges['tp_ratio']:
            current_iteration += 1
            
            try:
                # Calculate inputs and signals
                df_copy = df.copy()
                df_copy = calculate_enhanced_inputs(df_copy)
                df_copy = generate_enhanced_signals(df_copy, atr_sl, tp_ratio)
                
                # Simulate trades
                trades = simulate_enhanced_trades(df_copy)
                
                if len(trades) > 0:
                    # Calculate performance metrics
                    metrics = calculate_advanced_metrics(trades, STARTING_BALANCE)
                    metrics.update({
                        'atr_sl_multiplier': atr_sl,
                        'tp_ratio': tp_ratio,
                        'total_trades': len(trades)
                    })
                    
                    results.append(metrics)
                
                # Progress update
                progress = (current_iteration / total_iterations) * 100
                print(f"Progress: {progress:.1f}% - ATR_SL: {atr_sl}, TP: {tp_ratio}")
                
            except Exception as e:
                print(f"Error with parameters ATR_SL={atr_sl}, TP={tp_ratio}: {e}")
                continue
    
    return pd.DataFrame(results)

def calculate_advanced_metrics(trades, initial_balance):
    """Calculate advanced performance metrics"""
    
    if len(trades) == 0:
        return {
            'total_return': 0,
            'sharpe_ratio': 0,
            'max_drawdown': 0,
            'win_rate': 0,
            'profit_factor': 0,
            'avg_win': 0,
            'avg_loss': 0
        }
    
    # Basic metrics
    final_balance = trades['balance'].iloc[-1]
    total_return = (final_balance - initial_balance) / initial_balance
    
    # Daily returns for Sharpe ratio
    trades['daily_return'] = trades['balance'].pct_change().fillna(0)
    sharpe_ratio = trades['daily_return'].mean() / trades['daily_return'].std() * np.sqrt(365)
    
    # Maximum drawdown
    cumulative = trades['balance'].cummax()
    drawdown = (trades['balance'] - cumulative) / cumulative
    max_drawdown = drawdown.min()
    
    # Win rate and profit factor
    winning_trades = trades[trades['pnl'] > 0]
    losing_trades = trades[trades['pnl'] < 0]
    
    win_rate = len(winning_trades) / len(trades) if len(trades) > 0 else 0
    
    total_wins = winning_trades['pnl'].sum() if len(winning_trades) > 0 else 0
    total_losses = abs(losing_trades['pnl'].sum()) if len(losing_trades) > 0 else 0
    profit_factor = total_wins / total_losses if total_losses != 0 else float('inf')
    
    # Average win/loss
    avg_win = winning_trades['pnl'].mean() if len(winning_trades) > 0 else 0
    avg_loss = losing_trades['pnl'].mean() if len(losing_trades) > 0 else 0
    
    return {
        'total_return': total_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'win_rate': win_rate,
        'profit_factor': profit_factor,
        'avg_win': avg_win,
        'avg_loss': avg_loss
    }

print("Parameter optimization functions defined!")

## Run Strategy Optimization

Execute the parameter optimization:

In [ ]:
# Prepare data for optimization
df_optimized = price_data.copy()

# Run optimization
optimization_results = optimize_strategy_parameters(df_optimized)

# Display results
if len(optimization_results) > 0:
    print("\nOptimization Results:")
    print(optimization_results.sort_values('sharpe_ratio', ascending=False).head(10))
    
    # Find best parameters
    best_params = optimization_results.loc[optimization_results['sharpe_ratio'].idxmax()]
    print(f"\nBest Parameters:")
    print(f"ATR Stop Loss Multiplier: {best_params['atr_sl_multiplier']}")
    print(f"Take Profit Ratio: {best_params['tp_ratio']}")
    print(f"Sharpe Ratio: {best_params['sharpe_ratio']:.4f}")
    print(f"Total Return: {best_params['total_return']:.4f}")
    print(f"Max Drawdown: {best_params['max_drawdown']:.4f}")
    print(f"Win Rate: {best_params['win_rate']:.4f}")
else:
    print("No optimization results generated.")

## Backtest with Best Parameters

Run final backtest with optimized parameters:

In [ ]:
def run_final_backtest(df, best_params):
    """Run final backtest with optimized parameters"""
    
    # Calculate inputs and signals with best parameters
    df_final = df.copy()
    df_final = calculate_enhanced_inputs(df_final)
    df_final = generate_enhanced_signals(
        df_final, 
        best_params['atr_sl_multiplier'], 
        best_params['tp_ratio']
    )
    
    # Simulate trades
    final_trades = simulate_enhanced_trades(df_final)
    
    if len(final_trades) > 0:
        # Calculate final metrics
        final_metrics = calculate_advanced_metrics(final_trades, STARTING_BALANCE)
        
        print("\n=== FINAL BACKTEST RESULTS ===")
        print(f"Total Trades: {len(final_trades)}")
        print(f"Final Balance: ${final_trades['balance'].iloc[-1]:.2f}")
        print(f"Total Return: {final_metrics['total_return']:.4f}")
        print(f"Sharpe Ratio: {final_metrics['sharpe_ratio']:.4f}")
        print(f"Max Drawdown: {final_metrics['max_drawdown']:.4f}")
        print(f"Win Rate: {final_metrics['win_rate']:.4f}")
        print(f"Profit Factor: {final_metrics['profit_factor']:.4f}")
        print(f"Avg Win: ${final_metrics['avg_win']:.2f}")
        print(f"Avg Loss: ${final_metrics['avg_loss']:.2f}")
        
        return final_trades, final_metrics
    
    return None, None

# Run final backtest
if len(optimization_results) > 0:
    best_params = optimization_results.loc[optimization_results['sharpe_ratio'].idxmax()]
    final_trades, final_metrics = run_final_backtest(df_optimized, best_params)
else:
    print("No optimization results available for final backtest.")

## Performance Visualization

Create comprehensive performance visualizations:

In [ ]:
def create_performance_visualizations(trades, metrics):
    """Create comprehensive performance visualizations"""
    
    if trades is None or len(trades) == 0:
        print("No trades data available for visualization.")
        return
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('BTCUSD ORB+ATR Strategy Performance Analysis', fontsize=16)
    
    # 1. Balance over time
    axes[0, 0].plot(trades['entry_date'], trades['balance'], linewidth=2, color='cyan')
    axes[0, 0].set_title('Balance Over Time')
    axes[0, 0].set_xlabel('Date')
    axes[0, 0].set_ylabel('Balance ($)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Trade P&L distribution
    axes[0, 1].hist(trades['pnl'], bins=50, alpha=0.7, color='orange', edgecolor='black')
    axes[0, 1].axvline(trades['pnl'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: ${trades["pnl"].mean():.2f}')
    axes[0, 1].set_title('Trade P&L Distribution')
    axes[0, 1].set_xlabel('P&L ($)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Cumulative returns
    trades['cumulative_return'] = (trades['balance'] - STARTING_BALANCE) / STARTING_BALANCE
    axes[1, 0].plot(trades['entry_date'], trades['cumulative_return'], linewidth=2, color='green')
    axes[1, 0].set_title('Cumulative Returns')
    axes[1, 0].set_xlabel('Date')
    axes[1, 0].set_ylabel('Cumulative Return')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Monthly performance
    trades['month'] = pd.to_datetime(trades['entry_date']).dt.to_period('M')
    monthly_perf = trades.groupby('month')['pnl'].sum()
    monthly_perf.plot(kind='bar', ax=axes[1, 1], color='purple', alpha=0.7)
    axes[1, 1].set_title('Monthly Performance')
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_ylabel('P&L ($)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../data/btcusd_strategy_performance.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Additional analysis
    print("\n=== ADDITIONAL ANALYSIS ===")
    
    # Hourly performance
    trades['hour'] = pd.to_datetime(trades['entry_date']).dt.hour
    hourly_perf = trades.groupby('hour')['pnl'].agg(['mean', 'count'])
    print("\nHourly Performance:")
    print(hourly_perf.sort_values('mean', ascending=False).head())
    
    # Risk metrics
    print(f"\nRisk Metrics:")
    print(f"Calmar Ratio: {metrics['total_return'] / abs(metrics['max_drawdown']):.4f}")
    print(f"Sortino Ratio: {calculate_sortino_ratio(trades):.4f}")
    
def calculate_sortino_ratio(trades):
    """Calculate Sortino ratio (downside deviation based)"""
    daily_returns = trades['balance'].pct_change().dropna()
    downside_returns = daily_returns[daily_returns < 0]
    
    if len(downside_returns) == 0:
        return 0
    
    expected_return = daily_returns.mean()
    downside_std = downside_returns.std()
    
    if downside_std == 0:
        return 0
    
    return expected_return / downside_std * np.sqrt(365)

# Create visualizations
if final_trades is not None:
    create_performance_visualizations(final_trades, final_metrics)

## Strategy Validation and Robustness Testing

Perform robustness testing and validation:

In [ ]:
def perform_robustness_testing(df, best_params, num_tests=10):
    """Perform robustness testing with different market conditions"""
    
    print(f"Performing robustness testing with {num_tests} different scenarios...")
    
    robustness_results = []
    
    for i in range(num_tests):
        try:
            # Create different market conditions by sampling
            sample_size = int(len(df) * 0.8)  # Use 80% of data
            sample_df = df.sample(n=sample_size, random_state=i)
            
            # Run backtest on sample
            trades, metrics = run_final_backtest(sample_df, best_params)
            
            if trades is not None:
                robustness_results.append({
                    'test_number': i + 1,
                    'total_return': metrics['total_return'],
                    'sharpe_ratio': metrics['sharpe_ratio'],
                    'max_drawdown': metrics['max_drawdown'],
                    'win_rate': metrics['win_rate'],
                    'total_trades': len(trades)
                })
                
        except Exception as e:
            print(f"Error in robustness test {i+1}: {e}")
            continue
    
    if robustness_results:
        robustness_df = pd.DataFrame(robustness_results)
        
        print("\n=== ROBUSTNESS TESTING RESULTS ===")
        print(f"Average Total Return: {robustness_df['total_return'].mean():.4f}")
        print(f"Return Std Dev: {robustness_df['total_return'].std():.4f}")
        print(f"Average Sharpe Ratio: {robustness_df['sharpe_ratio'].mean():.4f}")
        print(f"Average Max Drawdown: {robustness_df['max_drawdown'].mean():.4f}")
        print(f"Average Win Rate: {robustness_df['win_rate'].mean():.4f}")
        
        # Plot robustness results
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle('Strategy Robustness Analysis', fontsize=16)
        
        # Total return distribution
        axes[0, 0].hist(robustness_df['total_return'], bins=10, alpha=0.7, color='blue')
        axes[0, 0].axvline(robustness_df['total_return'].mean(), color='red', linestyle='--', linewidth=2)
        axes[0, 0].set_title('Total Return Distribution')
        axes[0, 0].set_xlabel('Total Return')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Sharpe ratio distribution
        axes[0, 1].hist(robustness_df['sharpe_ratio'], bins=10, alpha=0.7, color='green')
        axes[0, 1].axvline(robustness_df['sharpe_ratio'].mean(), color='red', linestyle='--', linewidth=2)
        axes[0, 1].set_title('Sharpe Ratio Distribution')
        axes[0, 1].set_xlabel('Sharpe Ratio')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Max drawdown distribution
        axes[1, 0].hist(robustness_df['max_drawdown'], bins=10, alpha=0.7, color='red')
        axes[1, 0].axvline(robustness_df['max_drawdown'].mean(), color='black', linestyle='--', linewidth=2)
        axes[1, 0].set_title('Max Drawdown Distribution')
        axes[1, 0].set_xlabel('Max Drawdown')
        axes[1, 0].grid(True, alpha=0.3)
        
        # Win rate distribution
        axes[1, 1].hist(robustness_df['win_rate'], bins=10, alpha=0.7, color='purple')
        axes[1, 1].axvline(robustness_df['win_rate'].mean(), color='black', linestyle='--', linewidth=2)
        axes[1, 1].set_title('Win Rate Distribution')
        axes[1, 1].set_xlabel('Win Rate')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('../data/btcusd_robustness_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        return robustness_df
    
    return None

# Perform robustness testing
if len(optimization_results) > 0:
    best_params = optimization_results.loc[optimization_results['sharpe_ratio'].idxmax()]
    robustness_results = perform_robustness_testing(df_optimized, best_params)

## Summary and Recommendations

Generate final summary and trading recommendations:

In [ ]:
def generate_strategy_summary():
    """Generate comprehensive strategy summary"""
    
    print("=== BTCUSD ORB+ATR OPTIMIZED STRATEGY SUMMARY ===")
    print("\n1. STRATEGY OVERVIEW:")
    print("   - Opening Range Breakout (ORB) strategy for BTCUSD")
    print("   - 15-minute timeframe optimization")
    print("   - 24/7 cryptocurrency market adaptation")
    print("   - Fixed ATR value: 1344 (as specified)")
    print("   - Enhanced risk management")
    
    if len(optimization_results) > 0:
        best_params = optimization_results.loc[optimization_results['sharpe_ratio'].idxmax()]
        
        print("\n2. OPTIMAL PARAMETERS:")
        print(f"   - ATR Stop Loss Multiplier: {best_params['atr_sl_multiplier']}")
        print(f"   - Take Profit Ratio: {best_params['tp_ratio']}")
        
        print("\n3. PERFORMANCE METRICS:")
        print(f"   - Total Return: {best_params['total_return']:.4f}")
        print(f"   - Sharpe Ratio: {best_params['sharpe_ratio']:.4f}")
        print(f"   - Max Drawdown: {best_params['max_drawdown']:.4f}")
        print(f"   - Win Rate: {best_params['win_rate']:.4f}")
        print(f"   - Profit Factor: {best_params['profit_factor']:.4f}")
        
        if final_trades is not None:
            print("\n4. TRADE STATISTICS:")
            print(f"   - Total Trades: {len(final_trades)}")
            print(f"   - Final Balance: ${final_trades['balance'].iloc[-1]:.2f}")
            print(f"   - Average Win: ${final_metrics['avg_win']:.2f}")
            print(f"   - Average Loss: ${final_metrics['avg_loss']:.2f}")
    
    print("\n5. RISK MANAGEMENT:")
    print(f"   - Risk per trade: {RISK_PER_TRADE*100}%")
    print(f"   - Max daily loss: {MAX_DAILY_LOSS*100}%")
    print(f"   - Max daily trades: {MAX_DAILY_TRADES}")
    print(f"   - Starting balance: ${STARTING_BALANCE}")
    
    print("\n6. TRANSACTION COSTS:")
    print(f"   - Maker fee: {MAKER_FEE*100}%")
    print(f"   - Taker fee: {TAKER_FEE*100}%")
    print(f"   - Spread cost: {SPREAD_COST*100}%")
    
    print("\n7. RECOMMENDATIONS:")
    print("   - Use optimized parameters for live trading")
    print("   - Implement strict risk management")
    print("   - Monitor strategy performance regularly")
    print("   - Consider market conditions and volatility")
    print("   - Test on paper trading before live deployment")
    
    print("\n8. FILES GENERATED:")
    print("   - ../data/BTCUSD_M15_processed.csv (processed data)")
    print("   - ../data/btcusd_strategy_performance.png (performance charts)")
    print("   - ../data/btcusd_robustness_analysis.png (robustness analysis)")
    
    print("\n=== END OF SUMMARY ===")

# Generate summary
generate_strategy_summary()